In [1]:
import numpy as np
import pandas as pd

# from sklearn import preprocessing
from sklearn.impute import SimpleImputer


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.compose import ColumnTransformer
#from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier


from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt 



import seaborn as sns



from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.svm import LinearSVC, SVC
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from scipy import stats

In [3]:
### Read the bank data 
bank_data = pd.read_csv("train_ctrUa4K.csv")

valid_data =  pd.read_csv("test_lAUu6dG.csv")


print(bank_data.shape)
print(valid_data.shape)

(614, 13)
(367, 12)


In [4]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               601 non-null object
Married              611 non-null object
Dependents           599 non-null object
Education            614 non-null object
Self_Employed        582 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [37]:
bank_data.head

<bound method NDFrame.head of      Gender Married Dependents     Education Self_Employed  ApplicantIncome  \
0      Male      No          0      Graduate            No           5849.0   
1      Male     Yes          1      Graduate            No           4583.0   
2      Male     Yes          0      Graduate           Yes           3000.0   
3      Male     Yes          0  Not Graduate            No           2583.0   
4      Male      No          0      Graduate            No           6000.0   
..      ...     ...        ...           ...           ...              ...   
609  Female      No          0      Graduate            No           2900.0   
610    Male     Yes         3+      Graduate            No           4106.0   
611    Male     Yes          1      Graduate            No           8072.0   
612    Male     Yes          2      Graduate            No           7583.0   
613  Female      No          0      Graduate           Yes           4583.0   

     CoapplicantIncom

In [5]:
bank_data.Credit_History = bank_data.Credit_History.astype('category')
valid_data.Credit_History = valid_data.Credit_History.astype('category')

In [6]:
bank_data = bank_data.drop('Loan_ID', axis = 1)
valid_data = valid_data.drop('Loan_ID', axis = 1)

In [7]:
#### Adding code using the categorical and numerical Columns
num_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount','Loan_Amount_Term']
cat_cols = bank_data.columns.difference(num_cols).tolist()

print("Numeric Columns ->", num_cols)
print("Categorical Columns ->", cat_cols)

Numeric Columns -> ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
Categorical Columns -> ['Credit_History', 'Dependents', 'Education', 'Gender', 'Loan_Status', 'Married', 'Property_Area', 'Self_Employed']


In [8]:
for col in cat_cols:
    bank_data[col] = bank_data[col].astype('category')

In [9]:
cat_cols2 = ['Credit_History', 'Dependents', 'Education', 'Gender', 'Married', 'Property_Area', 'Self_Employed']
for col in cat_cols2:
    valid_data[col] = valid_data[col].astype('category')

In [10]:
 bank_data.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term
count,614.000000,614.000000,592.000000,600.00000
mean,5403.459283,1621.245798,146.412162,342.00000
std,6109.041673,2926.248369,85.587325,65.12041
min,150.000000,0.000000,9.000000,12.00000
25%,2877.500000,0.000000,100.000000,360.00000
50%,3812.500000,1188.500000,128.000000,360.00000
75%,5795.000000,2297.250000,168.000000,360.00000
max,81000.000000,41667.000000,700.000000,480.00000


In [11]:
valid_data.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term
count,367.000000,367.000000,362.000000,361.000000
mean,4805.599455,1569.577657,136.132597,342.537396
std,4910.685399,2334.232099,61.366652,65.156643
min,0.000000,0.000000,28.000000,6.000000
25%,2864.000000,0.000000,100.250000,360.000000
50%,3786.000000,1025.000000,125.000000,360.000000
75%,5060.000000,2430.500000,158.000000,360.000000
max,72529.000000,24000.000000,550.000000,480.000000


In [12]:
bank_data.isnull().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [13]:
valid_data.isnull().sum()

Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

##handling outliers

In [14]:
def getUpperLowerQuantiles(data) :
    LowerQuantile=[]
    UpperQuantile=[]
    num_cols=data.select_dtypes(include=['int64','float64']).columns
    print(num_cols)
    for i in num_cols:
        LowerQuantile.append(data[i].quantile(.01))
        UpperQuantile.append(data[i].quantile(.99))
    return(LowerQuantile,UpperQuantile)

In [15]:
LowerQuantile,UpperQuantile=list(getUpperLowerQuantiles(bank_data[num_cols]))

Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term'],
      dtype='object')


In [16]:
LowerQuantile

[1025.0, 0.0, 30.0, 84.0]

In [17]:
UpperQuantile

[32540.410000000044, 8895.890000000003, 496.3600000000001, 480.0]

In [18]:
def getOutliers(data) :
    for i in range(data.shape[1]):
        print("--------------")
        print(data.iloc[:,i][data.iloc[:,i]>UpperQuantile[i]])
        print(data.iloc[:,i][data.iloc[:,i]<LowerQuantile[i]])

In [19]:
getOutliers(bank_data[num_cols])

--------------
155    39999
171    51763
183    33846
185    39147
333    63337
409    81000
443    37719
Name: ApplicantIncome, dtype: int64
77     1000
188     674
216     150
468     210
500     645
600     416
Name: ApplicantIncome, dtype: int64
--------------
9      10968.0
122     8980.0
177    11300.0
402    20000.0
417    20000.0
581    33837.0
600    41667.0
Name: CoapplicantIncome, dtype: float64
Series([], Name: CoapplicantIncome, dtype: float64)
--------------
130    650.0
155    600.0
171    700.0
369    570.0
487    500.0
561    600.0
Name: LoanAmount, dtype: float64
14     17.0
94     25.0
133    25.0
555    26.0
568     9.0
Name: LoanAmount, dtype: float64
--------------
Series([], Name: Loan_Amount_Term, dtype: float64)
68     60.0
242    60.0
262    36.0
497    12.0
546    36.0
Name: Loan_Amount_Term, dtype: float64


In [20]:
def clip_outliers(data):
    data.clip(lower=LowerQuantile, upper=UpperQuantile, axis=1, inplace=True)
    return data

In [21]:
bank_data[num_cols] = clip_outliers(bank_data[num_cols])

In [22]:
getOutliers(bank_data[num_cols])

--------------
Series([], Name: ApplicantIncome, dtype: float64)
Series([], Name: ApplicantIncome, dtype: float64)
--------------
Series([], Name: CoapplicantIncome, dtype: float64)
Series([], Name: CoapplicantIncome, dtype: float64)
--------------
Series([], Name: LoanAmount, dtype: float64)
Series([], Name: LoanAmount, dtype: float64)
--------------
Series([], Name: Loan_Amount_Term, dtype: float64)
Series([], Name: Loan_Amount_Term, dtype: float64)


In [23]:
valid_outliers_data = valid_data.copy()


In [24]:
valid_outliers_data[num_cols] = clip_outliers(valid_outliers_data[num_cols])


In [25]:
getOutliers(valid_outliers_data[num_cols])

--------------
Series([], Name: ApplicantIncome, dtype: float64)
Series([], Name: ApplicantIncome, dtype: float64)
--------------
Series([], Name: CoapplicantIncome, dtype: float64)
Series([], Name: CoapplicantIncome, dtype: float64)
--------------
Series([], Name: LoanAmount, dtype: float64)
Series([], Name: LoanAmount, dtype: float64)
--------------
Series([], Name: Loan_Amount_Term, dtype: float64)
Series([], Name: Loan_Amount_Term, dtype: float64)


In [26]:
##handling missing values

In [27]:
bank_data.isnull().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [28]:
valid_outliers_data.isnull().sum()

Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

In [29]:
 bank_data.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term
count,614.000000,614.000000,592.000000,600.000000
mean,5214.577964,1483.654967,145.409054,342.360000
std,4528.770082,1831.226927,79.927273,63.531074
min,1025.000000,0.000000,30.000000,84.000000
25%,2877.500000,0.000000,100.000000,360.000000
50%,3812.500000,1188.500000,128.000000,360.000000
75%,5795.000000,2297.250000,168.000000,360.000000
max,32540.410000,8895.890000,496.360000,480.000000


In [30]:
valid_outliers_data.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term
count,367.000000,367.000000,362.000000,361.000000
mean,4704.284496,1475.091144,135.995470,343.152355
std,3689.227473,1774.432923,60.402421,62.344576
min,1025.000000,0.000000,30.000000,84.000000
25%,2864.000000,0.000000,100.250000,360.000000
50%,3786.000000,1025.000000,125.000000,360.000000
75%,5060.000000,2430.500000,158.000000,360.000000
max,32540.410000,8895.890000,496.360000,480.000000


In [31]:

# impute num cols with mean
median_imputer = SimpleImputer(strategy='mean')
median_imputer.fit(bank_data[num_cols])

bank_data[num_cols] = median_imputer.transform(bank_data[num_cols])
valid_outliers_data[num_cols] = median_imputer.transform(valid_outliers_data[num_cols])

valid_data[num_cols] = median_imputer.transform(valid_data[num_cols])

In [34]:
pd.DataFrame({"Train":bank_data.isnull().sum(),"valid_data":valid_data.isnull().sum(),"valid_outliers_data":valid_outliers_data.isnull().sum()})

,Train,valid_data,valid_outliers_data
ApplicantIncome,0,0.0,0.0
CoapplicantIncome,0,0.0,0.0
Credit_History,50,29.0,29.0
Dependents,15,10.0,10.0
Education,0,0.0,0.0
Gender,13,11.0,11.0
LoanAmount,0,0.0,0.0
Loan_Amount_Term,0,0.0,0.0
Loan_Status,0,NaN,NaN
Married,3,0.0,0.0


In [35]:
# impute cat cols missings with mode
cat_cols2 = ['Credit_History', 'Dependents', 'Education', 'Gender','Married', 'Property_Area', 'Self_Employed']


median_imputer = SimpleImputer(strategy='most_frequent')
median_imputer.fit(bank_data[cat_cols2])

bank_data[cat_cols2] = median_imputer.transform(bank_data[cat_cols2])
valid_outliers_data[cat_cols2] = median_imputer.transform(valid_outliers_data[cat_cols2])

valid_data[cat_cols2] = median_imputer.transform(valid_data[cat_cols2])

In [36]:
pd.DataFrame({"Train":bank_data.isnull().sum(),"valid_data":valid_data.isnull().sum(),"valid_outliers_data":valid_outliers_data.isnull().sum()})

,Train,valid_data,valid_outliers_data
ApplicantIncome,0,0.0,0.0
CoapplicantIncome,0,0.0,0.0
Credit_History,0,0.0,0.0
Dependents,0,0.0,0.0
Education,0,0.0,0.0
Gender,0,0.0,0.0
LoanAmount,0,0.0,0.0
Loan_Amount_Term,0,0.0,0.0
Loan_Status,0,NaN,NaN
Married,0,0.0,0.0


In [137]:
y = np.array(bank_data["Loan_Status"])
Credit_History_encoder = LabelEncoder()
bank_data['Credit_History'] =  Credit_History_encoder.fit_transform(bank_data["Credit_History"])
Dependents_encoder = LabelEncoder()
bank_data['Dependents'] = Dependents_encoder.fit_transform(bank_data["Dependents"])
Education_encoder = LabelEncoder()
bank_data['Education'] = Education_encoder.fit_transform(bank_data["Education"])
Gender_encoder = OneHotEncoder(dtype=np.uint8)
Gender = pd.DataFrame(Gender_encoder.fit_transform(bank_data["Gender"].values.reshape(-1,1)).todense())
Married_encoder = OneHotEncoder(dtype=np.uint8)
Married = pd.DataFrame(Married_encoder.fit_transform(bank_data["Married"].values.reshape(-1,1)).todense())
Property_Area_encoder = LabelEncoder()
bank_data['Property_Area'] = Property_Area_encoder.fit_transform(bank_data["Property_Area"])
Self_Employed_encoder = LabelEncoder()
bank_data['Self_Employed'] = Self_Employed_encoder.fit_transform(bank_data["Self_Employed"])

X = bank_data[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term','Credit_History','Dependents','Education',
              'Property_Area','Self_Employed']]

X = pd.concat((X,Gender,Married), axis = 1)
X = np.array(X)

In [141]:
X 

array([[5.84900000e+03, 0.00000000e+00, 1.45409054e+02, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [4.58300000e+03, 1.50800000e+03, 1.28000000e+02, ...,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [3.00000000e+03, 0.00000000e+00, 6.60000000e+01, ...,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       ...,
       [8.07200000e+03, 2.40000000e+02, 2.53000000e+02, ...,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [7.58300000e+03, 0.00000000e+00, 1.87000000e+02, ...,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [4.58300000e+03, 0.00000000e+00, 1.33000000e+02, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [142]:

models = []
fold = StratifiedKFold(n_splits=5)
for train_index, test_index in fold.split(X,y):
    X_train,X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #model = CatBoostClassifier(iterations=2000,learning_rate=0.015, verbose=0)
    model = CatBoostClassifier(verbose=1)
    model.fit(X_train,y_train)
    print (model.score(X_test, y_test))
    models.append(model)

Learning rate set to 0.007604
0:	learn: 0.6884247	total: 3.86ms	remaining: 3.85s
1:	learn: 0.6847034	total: 6.08ms	remaining: 3.03s
2:	learn: 0.6811404	total: 9.69ms	remaining: 3.22s
3:	learn: 0.6773547	total: 12.6ms	remaining: 3.15s
4:	learn: 0.6735945	total: 17.4ms	remaining: 3.46s
5:	learn: 0.6696888	total: 21.5ms	remaining: 3.56s
6:	learn: 0.6659666	total: 24.1ms	remaining: 3.41s
7:	learn: 0.6627509	total: 25.5ms	remaining: 3.15s
8:	learn: 0.6589296	total: 27.9ms	remaining: 3.07s
9:	learn: 0.6552950	total: 31.3ms	remaining: 3.09s
10:	learn: 0.6520426	total: 32.9ms	remaining: 2.95s
11:	learn: 0.6486346	total: 35.7ms	remaining: 2.94s
12:	learn: 0.6454272	total: 38.1ms	remaining: 2.9s
13:	learn: 0.6422295	total: 40.6ms	remaining: 2.86s
14:	learn: 0.6387375	total: 43ms	remaining: 2.82s
15:	learn: 0.6355693	total: 46.4ms	remaining: 2.86s
16:	learn: 0.6322739	total: 49.3ms	remaining: 2.85s
17:	learn: 0.6291922	total: 51.7ms	remaining: 2.82s
18:	learn: 0.6262125	total: 54.2ms	remaining: 2

187:	learn: 0.4220184	total: 511ms	remaining: 2.21s
188:	learn: 0.4216992	total: 515ms	remaining: 2.21s
189:	learn: 0.4212607	total: 518ms	remaining: 2.21s
190:	learn: 0.4208395	total: 520ms	remaining: 2.2s
191:	learn: 0.4204402	total: 523ms	remaining: 2.2s
192:	learn: 0.4197847	total: 526ms	remaining: 2.2s
193:	learn: 0.4192082	total: 529ms	remaining: 2.2s
194:	learn: 0.4186739	total: 532ms	remaining: 2.19s
195:	learn: 0.4184649	total: 534ms	remaining: 2.19s
196:	learn: 0.4180166	total: 538ms	remaining: 2.19s
197:	learn: 0.4175577	total: 541ms	remaining: 2.19s
198:	learn: 0.4170553	total: 544ms	remaining: 2.19s
199:	learn: 0.4166928	total: 547ms	remaining: 2.19s
200:	learn: 0.4161852	total: 551ms	remaining: 2.19s
201:	learn: 0.4157529	total: 555ms	remaining: 2.19s
202:	learn: 0.4151175	total: 557ms	remaining: 2.19s
203:	learn: 0.4144972	total: 560ms	remaining: 2.18s
204:	learn: 0.4143105	total: 562ms	remaining: 2.18s
205:	learn: 0.4139960	total: 566ms	remaining: 2.18s
206:	learn: 0.41

375:	learn: 0.3543311	total: 1.03s	remaining: 1.71s
376:	learn: 0.3541252	total: 1.03s	remaining: 1.7s
377:	learn: 0.3538652	total: 1.03s	remaining: 1.7s
378:	learn: 0.3536159	total: 1.04s	remaining: 1.7s
379:	learn: 0.3533838	total: 1.04s	remaining: 1.7s
380:	learn: 0.3531645	total: 1.04s	remaining: 1.7s
381:	learn: 0.3528753	total: 1.05s	remaining: 1.69s
382:	learn: 0.3526438	total: 1.05s	remaining: 1.69s
383:	learn: 0.3522567	total: 1.05s	remaining: 1.69s
384:	learn: 0.3521068	total: 1.06s	remaining: 1.69s
385:	learn: 0.3517732	total: 1.06s	remaining: 1.69s
386:	learn: 0.3515259	total: 1.06s	remaining: 1.68s
387:	learn: 0.3511726	total: 1.06s	remaining: 1.68s
388:	learn: 0.3509131	total: 1.07s	remaining: 1.68s
389:	learn: 0.3506984	total: 1.07s	remaining: 1.67s
390:	learn: 0.3505258	total: 1.07s	remaining: 1.67s
391:	learn: 0.3503628	total: 1.07s	remaining: 1.67s
392:	learn: 0.3500224	total: 1.08s	remaining: 1.66s
393:	learn: 0.3496940	total: 1.08s	remaining: 1.66s
394:	learn: 0.349

561:	learn: 0.3070997	total: 1.54s	remaining: 1.2s
562:	learn: 0.3068171	total: 1.55s	remaining: 1.2s
563:	learn: 0.3064995	total: 1.55s	remaining: 1.2s
564:	learn: 0.3062795	total: 1.55s	remaining: 1.2s
565:	learn: 0.3059729	total: 1.56s	remaining: 1.19s
566:	learn: 0.3057824	total: 1.56s	remaining: 1.19s
567:	learn: 0.3054662	total: 1.56s	remaining: 1.19s
568:	learn: 0.3052194	total: 1.56s	remaining: 1.19s
569:	learn: 0.3048840	total: 1.57s	remaining: 1.18s
570:	learn: 0.3047009	total: 1.57s	remaining: 1.18s
571:	learn: 0.3044870	total: 1.57s	remaining: 1.18s
572:	learn: 0.3043518	total: 1.58s	remaining: 1.18s
573:	learn: 0.3042795	total: 1.58s	remaining: 1.17s
574:	learn: 0.3040924	total: 1.58s	remaining: 1.17s
575:	learn: 0.3039527	total: 1.58s	remaining: 1.17s
576:	learn: 0.3036339	total: 1.59s	remaining: 1.16s
577:	learn: 0.3034511	total: 1.59s	remaining: 1.16s
578:	learn: 0.3034155	total: 1.59s	remaining: 1.16s
579:	learn: 0.3032461	total: 1.59s	remaining: 1.16s
580:	learn: 0.30

757:	learn: 0.2665133	total: 2.06s	remaining: 657ms
758:	learn: 0.2663865	total: 2.06s	remaining: 655ms
759:	learn: 0.2662088	total: 2.06s	remaining: 652ms
760:	learn: 0.2660369	total: 2.07s	remaining: 649ms
761:	learn: 0.2659150	total: 2.07s	remaining: 647ms
762:	learn: 0.2657085	total: 2.07s	remaining: 644ms
763:	learn: 0.2655671	total: 2.08s	remaining: 641ms
764:	learn: 0.2653106	total: 2.08s	remaining: 639ms
765:	learn: 0.2650715	total: 2.08s	remaining: 636ms
766:	learn: 0.2649339	total: 2.08s	remaining: 633ms
767:	learn: 0.2647700	total: 2.09s	remaining: 631ms
768:	learn: 0.2645354	total: 2.09s	remaining: 628ms
769:	learn: 0.2643924	total: 2.09s	remaining: 626ms
770:	learn: 0.2642048	total: 2.1s	remaining: 623ms
771:	learn: 0.2640536	total: 2.1s	remaining: 620ms
772:	learn: 0.2639312	total: 2.1s	remaining: 617ms
773:	learn: 0.2637860	total: 2.1s	remaining: 614ms
774:	learn: 0.2636305	total: 2.11s	remaining: 612ms
775:	learn: 0.2634606	total: 2.11s	remaining: 609ms
776:	learn: 0.26

...

188:	learn: 0.4141213	total: 667ms	remaining: 2.86s
189:	learn: 0.4137055	total: 670ms	remaining: 2.86s
190:	learn: 0.4136380	total: 672ms	remaining: 2.84s
191:	learn: 0.4132457	total: 674ms	remaining: 2.84s
192:	learn: 0.4126706	total: 677ms	remaining: 2.83s
193:	learn: 0.4124312	total: 680ms	remaining: 2.83s
194:	learn: 0.4119130	total: 683ms	remaining: 2.82s
195:	learn: 0.4117590	total: 684ms	remaining: 2.81s
196:	learn: 0.4111945	total: 687ms	remaining: 2.8s
197:	learn: 0.4105384	total: 690ms	remaining: 2.79s
198:	learn: 0.4101384	total: 693ms	remaining: 2.79s
199:	learn: 0.4098104	total: 695ms	remaining: 2.78s
200:	learn: 0.4094039	total: 699ms	remaining: 2.78s
201:	learn: 0.4090253	total: 702ms	remaining: 2.77s
202:	learn: 0.4082353	total: 704ms	remaining: 2.76s
203:	learn: 0.4077200	total: 707ms	remaining: 2.76s
204:	learn: 0.4071687	total: 709ms	remaining: 2.75s
205:	learn: 0.4065144	total: 712ms	remaining: 2.74s
206:	learn: 0.4061881	total: 714ms	remaining: 2.74s
207:	learn: 0

398:	learn: 0.3408980	total: 1.18s	remaining: 1.78s
399:	learn: 0.3406585	total: 1.19s	remaining: 1.78s
400:	learn: 0.3401736	total: 1.19s	remaining: 1.78s
401:	learn: 0.3400186	total: 1.19s	remaining: 1.77s
402:	learn: 0.3396569	total: 1.2s	remaining: 1.77s
403:	learn: 0.3392283	total: 1.2s	remaining: 1.77s
404:	learn: 0.3390852	total: 1.2s	remaining: 1.76s
405:	learn: 0.3388758	total: 1.2s	remaining: 1.76s
406:	learn: 0.3385797	total: 1.21s	remaining: 1.76s
407:	learn: 0.3385381	total: 1.21s	remaining: 1.75s
408:	learn: 0.3382208	total: 1.21s	remaining: 1.75s
409:	learn: 0.3379756	total: 1.22s	remaining: 1.75s
410:	learn: 0.3377103	total: 1.22s	remaining: 1.75s
411:	learn: 0.3373367	total: 1.22s	remaining: 1.75s
412:	learn: 0.3370586	total: 1.23s	remaining: 1.74s
413:	learn: 0.3368833	total: 1.23s	remaining: 1.74s
414:	learn: 0.3366721	total: 1.23s	remaining: 1.74s
415:	learn: 0.3363211	total: 1.23s	remaining: 1.73s
416:	learn: 0.3360310	total: 1.24s	remaining: 1.73s
417:	learn: 0.33

600:	learn: 0.2929064	total: 1.69s	remaining: 1.12s
601:	learn: 0.2924524	total: 1.69s	remaining: 1.12s
602:	learn: 0.2923011	total: 1.69s	remaining: 1.11s
603:	learn: 0.2921167	total: 1.69s	remaining: 1.11s
604:	learn: 0.2918797	total: 1.7s	remaining: 1.11s
605:	learn: 0.2917715	total: 1.7s	remaining: 1.1s
606:	learn: 0.2916403	total: 1.7s	remaining: 1.1s
607:	learn: 0.2913897	total: 1.71s	remaining: 1.1s
608:	learn: 0.2912485	total: 1.71s	remaining: 1.1s
609:	learn: 0.2910277	total: 1.71s	remaining: 1.09s
610:	learn: 0.2907842	total: 1.72s	remaining: 1.09s
611:	learn: 0.2906781	total: 1.72s	remaining: 1.09s
612:	learn: 0.2905059	total: 1.72s	remaining: 1.09s
613:	learn: 0.2903503	total: 1.72s	remaining: 1.08s
614:	learn: 0.2901605	total: 1.73s	remaining: 1.08s
615:	learn: 0.2900248	total: 1.73s	remaining: 1.08s
616:	learn: 0.2897458	total: 1.73s	remaining: 1.07s
617:	learn: 0.2895045	total: 1.73s	remaining: 1.07s
618:	learn: 0.2893400	total: 1.74s	remaining: 1.07s
619:	learn: 0.28911

804:	learn: 0.2519430	total: 2.2s	remaining: 533ms
805:	learn: 0.2516593	total: 2.21s	remaining: 531ms
806:	learn: 0.2515859	total: 2.21s	remaining: 529ms
807:	learn: 0.2513833	total: 2.21s	remaining: 526ms
808:	learn: 0.2512570	total: 2.21s	remaining: 523ms
809:	learn: 0.2511429	total: 2.22s	remaining: 520ms
810:	learn: 0.2509343	total: 2.22s	remaining: 518ms
811:	learn: 0.2506893	total: 2.22s	remaining: 515ms
812:	learn: 0.2503551	total: 2.23s	remaining: 512ms
813:	learn: 0.2501956	total: 2.23s	remaining: 509ms
814:	learn: 0.2499407	total: 2.23s	remaining: 507ms
815:	learn: 0.2497141	total: 2.24s	remaining: 504ms
816:	learn: 0.2495787	total: 2.24s	remaining: 502ms
817:	learn: 0.2493569	total: 2.24s	remaining: 499ms
818:	learn: 0.2492947	total: 2.24s	remaining: 496ms
819:	learn: 0.2491237	total: 2.25s	remaining: 493ms
820:	learn: 0.2489985	total: 2.25s	remaining: 490ms
821:	learn: 0.2488047	total: 2.25s	remaining: 488ms
822:	learn: 0.2487029	total: 2.25s	remaining: 485ms
823:	learn: 0

0.819672131147541


In [143]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

print(confusion_matrix(y_true=y_train, y_pred = train_pred))

confusion_matrix_test = confusion_matrix(y_true=y_test, y_pred =  test_pred)


Accuracy_test=(confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])

Precision_Test= confusion_matrix_test[1,1]/(confusion_matrix_test[1,1]+confusion_matrix_test[0,1])
Recall_Test= confusion_matrix_test[1,1]/(confusion_matrix_test[1,0]+confusion_matrix_test[1,1])

print("Test Precision: ",Precision_Test)
print("Test Recall: ",Recall_Test)
print("Train Accuracy: ",Accuracy_test)

0.9288617886178862
0.819672131147541
[[120  34]
 [  1 337]]
Test Precision:  0.81
Test Recall:  0.9642857142857143
Train Accuracy:  0.819672131147541


In [144]:
for model in models:
    print (model.learning_rate_)

0.00760400015860796
0.00760400015860796
0.00760400015860796
0.00760400015860796
0.00761000020429492


In [148]:

Credit_History_encoder = LabelEncoder()
valid_data['Credit_History'] =  Credit_History_encoder.fit_transform(valid_data["Credit_History"])
Dependents_encoder = LabelEncoder()
valid_data['Dependents'] = Dependents_encoder.fit_transform(valid_data["Dependents"])
Education_encoder = LabelEncoder()
valid_data['Education'] = Education_encoder.fit_transform(valid_data["Education"])
Gender_encoder = OneHotEncoder(dtype=np.uint8)
Gender = pd.DataFrame(Gender_encoder.fit_transform(valid_data["Gender"].values.reshape(-1,1)).todense())
Married_encoder = OneHotEncoder(dtype=np.uint8)
Married = pd.DataFrame(Married_encoder.fit_transform(valid_data["Married"].values.reshape(-1,1)).todense())
Property_Area_encoder = LabelEncoder()
valid_data['Property_Area'] = Property_Area_encoder.fit_transform(valid_data["Property_Area"])
Self_Employed_encoder = LabelEncoder()
valid_data['Self_Employed'] = Self_Employed_encoder.fit_transform(valid_data["Self_Employed"])

X_valid = valid_data[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term','Credit_History','Dependents','Education',
              'Property_Area','Self_Employed']]

X_valid = pd.concat((X_valid,Gender,Married), axis = 1)
X_valid = np.array(X_valid)

In [157]:
y_valid = []
for model in models:
    y_valid.append(model.predict(X_valid))
y_valid = np.array(y)
print (y.shape)
#y = stats.mode(y,axis=0).mode.reshape(-1,)
(values,counts) = np.unique(y_valid,return_counts=True)
ind=np.argmax(counts)
#y_valid = y_valid[ind].reshape(-1,)

(367,)


In [151]:
df =  pd.read_csv("test_lAUu6dG.csv")

ids = df["Loan_ID"].values
with open("sub3.csv","w") as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(["Loan_ID", "Loan_Status"])
    i = 0
    for id in ids:
        csv_writer.writerow([id,y_valid[i]])
        i += 1

In [165]:
model_adaboost = []
fold = StratifiedKFold(n_splits=5)
for train_index, test_index in fold.split(X,y):
    X_train,X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    models_adaboost = AdaBoostClassifier(n_estimators=100, learning_rate=0.015)
    models_adaboost.fit(X_train,y_train)
    print (models_adaboost.score(X_test, y_test))
    model_adaboost.append(models_adaboost)

1.0
1.0
0.9863013698630136
0.9726027397260274
1.0


In [166]:
y_valid_adaboost = []
for model in models:
    y_valid_adaboost.append(models_adaboost.predict(X_valid))
y_valid_adaboost = np.array(y)
print (y_valid_adaboost.shape)
#y = stats.mode(y,axis=0).mode.reshape(-1,)
(values,counts) = np.unique(y_valid_adaboost,return_counts=True)
ind=np.argmax(counts)
#y_valid_adaboost = y_valid_adaboost[ind].reshape(-1,)

(367,)


In [169]:
df =  pd.read_csv("test_lAUu6dG.csv")

ids = df["Loan_ID"].values
with open("sub6_adaboost.csv","w") as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(["Loan_ID", "Loan_Status"])
    i = 0
    for id in ids:
        csv_writer.writerow([id,y_valid_adaboost[i]])
        i += 1